In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import json
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

import statsmodels.api as sm
from scipy import stats
from scipy.stats import ttest_1samp, ttest_rel, ttest_ind

### Getting Test Playlist From Spotify

In [2]:
username = 'britopereiraa'
client_id ='17a3ed14ab434b33b28f19f8bea3337d'
client_secret = '7f848f9d2a024b15be5fa776b4dc7a05'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [3]:
print(token)

BQDjzloNkoQuuLcB2DInXoilQoXgDPyHS11GyxFI0tBCtTzG-6s-zJntyXDbt-FkwCUZVyg0YWrQHYx0cf29ZXJOcq9kCw7NjfUdXIpbrA0_dEFrtGquTWsYkJimCpWgORls2Do3UVHIoeqTp52b8g-fyxI


In [4]:
credentials = json.load(open('authorization.json'))
client_id = credentials['client_id']
client_secret = credentials['client_secret']

#### Vodafone FM 

https://open.spotify.com/playlist/5QZyi299jBblzaTFW1nP4h?si=PawXR_5mSL-ZfezTCUX2Hg

In [5]:
#I'm going to test with a playlist from Vodafone FM 

playlist_index = 0

playlists = json.load(open('playlists_test.json'))
playlist_uri = playlists[playlist_index]['uri']
test = playlists[playlist_index]['like']

client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlist_uri

#just to confirm: spotify:playlist:3M2gjOU19KrxZOCYtVYUCY

'spotify:user:Test_1:playlist:3M2gjOU19KrxZOCYtVYUCY'

In [6]:
def master_function(uri):
    uri = playlist_uri    
    username = uri.split(':')[2]
    playlist_id = uri.split(':')[4]
    results = {'items':[]}

    for n in range(0,3000,100):
        new = sp.user_playlist_tracks(username, playlist_id,  offset = n)
        results['items'] += new['items']

        playlist_tracks_data = results
        playlist_tracks_id = []
        playlist_tracks_titles = []
        playlist_tracks_artists = []

        for track in playlist_tracks_data['items']:
            playlist_tracks_id.append(track['track']['id'])
            playlist_tracks_titles.append(track['track']['name'])

        #adds a list of all artists involved in the song to the list of artists for the playlist
            for artist in track['track']['artists']:
                artist_list = []
                artist_list.append(artist['name'])
            playlist_tracks_artists.append(artist_list[0])
    
    df = pd.DataFrame([])
    for i in range(0, len(playlist_tracks_id)):
        features = sp.audio_features(playlist_tracks_id[i])
        features_df = pd.DataFrame(features)
        df = df.append(features_df)

    df['title'] = playlist_tracks_titles
    #features_df['first_artist'] = playlist_tracks_first_artists
    df['main_artist'] = playlist_tracks_artists
    #features_df = features_df.set_index('id')
    df = df[['id', 'title', 'main_artist',
                               'danceability', 'energy', 'key', 'loudness',
                               'mode', 'acousticness', 'instrumentalness',
                               'liveness', 'valence', 'tempo',
                               'duration_ms', 'time_signature']]

    return df

In [7]:
test_df = master_function(playlist_uri)

In [8]:
test_df.shape

(292, 15)

#### Test Random Forest

In order to test my best classifier (Random Forest):

In [9]:
#using the best algorithm (have to train again...)

data = pd.read_csv('my_data.csv')

features = ['danceability', 'energy', 'key','loudness', 'mode', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature']

X = data[features]
y = data['Like']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1)

clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

metrics.accuracy_score(y_test, y_pred)

0.8641509433962264

In [36]:
#passing with the test playlist 

pred = clf.predict_proba(test_df[features])[:,1]

test_df['prediction'] = pred
test_df.head()

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,prediction
0,0HV1cGoX5ASBlPA1pqH8fG,Every Other Freckle,alt-J,0.528,0.707,2,-7.626,1,0.6250,0.03550,0.116,0.502,143.802,216004,4,0.94
0,07LpPO7qBMtJGZ3fz5Khxw,Hi-Five,Angel Olsen,0.514,0.574,9,-7.224,1,0.0443,0.00826,0.142,0.369,125.389,177520,4,0.64
0,6hlJIYn5IBNqcSGTS6X5Xq,My Girls,Animal Collective,0.508,0.953,4,-6.031,0,0.2560,0.04450,0.929,0.555,119.973,340787,4,0.46
0,6bZaumwiWjUSLSLqUyxetj,The Suburbs,Arcade Fire,0.618,0.676,6,-6.127,0,0.0577,0.01970,0.343,0.537,117.783,315200,4,0.66
0,4VAkPXhI7xQ6FHlUuY3RzB,Wake Up,Arcade Fire,0.254,0.787,0,-4.813,1,0.1110,0.04490,0.105,0.301,139.389,335333,4,0.46


In [38]:
print("How similar is it to my taste?", round(test_df['prediction'].mean()*100,3), "%")

How similar is it to my taste? 46.308 %


In [37]:
liked_test_songs = test_df[test_df['prediction'] >0.9]
liked_test_songs

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,prediction
0,0HV1cGoX5ASBlPA1pqH8fG,Every Other Freckle,alt-J,0.528,0.7070,2,-7.626,1,0.6250,0.035500,0.1160,0.5020,143.802,216004,4,0.94
0,70WyPf8WtXvG3CgeTZZWz6,Nantes,Beirut,0.541,0.5330,6,-9.575,1,0.7860,0.010200,0.1380,0.6420,131.147,230200,4,0.98
0,3Ay3OuDTieyXTNibBO0mBy,Red Diamond,Best Youth,0.569,0.3600,9,-10.418,0,0.6110,0.001130,0.1130,0.1730,118.983,256134,4,1.00
0,0kHoisnf1ddnV6MrFLWka9,Hang Out,Best Youth,0.514,0.4520,6,-10.420,0,0.1940,0.455000,0.1770,0.2000,139.885,264000,4,0.97
0,7fXyuQqEBwpKtCa8PomO7B,Skinny Love,Bon Iver,0.590,0.2600,4,-14.065,0,0.8500,0.000005,0.1170,0.1120,77.118,238520,4,0.95
0,4ccIlWJ2R0FAI8bDyFdMTu,Our Love,Caribou,0.729,0.4950,10,-10.747,0,0.1310,0.886000,0.1020,0.4830,125.002,334173,4,0.92
0,0iHwnccPp214om2eGT5gzm,1998,Chet Faker,0.785,0.3820,1,-13.033,1,0.3480,0.344000,0.0882,0.5080,99.983,365382,4,0.93
0,1MJ5f5EYBC92ADD6xcz7nb,Chamber Of Reflection,Mac DeMarco,0.537,0.5580,11,-8.678,1,0.2630,0.910000,0.1020,0.5050,131.037,231724,4,0.91
0,1QNUdBrQyfboTVqaf4R3hL,A pele que há em mim (Quando o dia entardeceu)...,JP Simões,0.637,0.1580,9,-13.508,1,0.9240,0.003620,0.1110,0.0919,104.502,234280,3,0.97
0,49mNpggwlrIeNQ3Ka4JlFy,Pra quem quer,Márcia,0.670,0.3540,4,-14.004,1,0.6330,0.393000,0.1490,0.3560,134.030,226253,4,0.97


### Test Felipe's taste 

In [11]:
#I'm going to test with a playlist from Felipe: 'Uma'

playlist_index = 1

playlists = json.load(open('playlists_test.json'))
playlist_uri = playlists[playlist_index]['uri']
test = playlists[playlist_index]['like']

client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlist_uri
#spotify:playlist:12Wbv8sIx84T5uh6iOoJ7V

'spotify:user:Test_2:playlist:12Wbv8sIx84T5uh6iOoJ7V'

In [14]:
felipe_df = master_function(playlist_uri)
felipe_df

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0A0eOcimSNRs2EQQlH7FFJ,Três Dias,Marcelo Camelo,0.291,0.319,7,-10.465,1,0.791000,0.468000,0.0692,0.038,103.793,323280,3
0,0MtVmhAx6CxNuxFIUc6Mj9,Elephant Gun,Beirut,0.171,0.626,8,-8.677,1,0.873000,0.025200,0.0681,0.457,180.098,348893,3
0,6aUAF8JOd8zEl41B6I18xL,Fake Empire,The National,0.324,0.776,0,-6.784,1,0.151000,0.917000,0.0728,0.317,101.964,205040,3
0,2vGvPQNnyybJmiqpr1HiKX,Sorriso ao Sono,Phill Veras,0.627,0.342,2,-12.833,1,0.700000,0.166000,0.0820,0.513,80.030,248480,4
0,1PuLHwFZoh5qYK89I5YBdZ,I Will Wait,Mumford & Sons,0.486,0.769,1,-5.140,1,0.011900,0.008320,0.1970,0.393,131.286,276720,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,6x14wL9HvABR2Rix8N88EH,Dylan Thomas,Conor Oberst,0.592,0.805,2,-7.995,1,0.000128,0.000211,0.0916,0.505,116.072,216787,4
0,6xMFDLtFIiObaAuB42qHGW,Hidden Edelweiss,Spark Alaska,0.758,0.347,9,-13.068,1,0.860000,0.005930,0.3020,0.272,143.443,220943,4
0,669PEr8I3wOswY8BANBpdh,Shaayad,Taba Chake,0.606,0.472,8,-8.525,1,0.685000,0.000011,0.1140,0.439,94.972,208000,4
0,65wICNTM2s1PwXSgKt8f0a,Kill All Your Friends,My Chemical Romance,0.472,0.902,7,-3.660,1,0.000825,0.000033,0.1210,0.493,102.564,268883,4


In [15]:
felipe_df.shape

(162, 15)

In [31]:
#passing with the Felipe's playlist 

pred = clf.predict_proba(felipe_df[features])[:,1]

felipe_df['prediction'] = pred
felipe_df.head()


,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,prediction
0,0A0eOcimSNRs2EQQlH7FFJ,Três Dias,Marcelo Camelo,0.291,0.319,7,-10.465,1,0.7910,0.46800,0.0692,0.038,103.793,323280,3,0.95
0,0MtVmhAx6CxNuxFIUc6Mj9,Elephant Gun,Beirut,0.171,0.626,8,-8.677,1,0.8730,0.02520,0.0681,0.457,180.098,348893,3,0.77
0,6aUAF8JOd8zEl41B6I18xL,Fake Empire,The National,0.324,0.776,0,-6.784,1,0.1510,0.91700,0.0728,0.317,101.964,205040,3,0.63
0,2vGvPQNnyybJmiqpr1HiKX,Sorriso ao Sono,Phill Veras,0.627,0.342,2,-12.833,1,0.7000,0.16600,0.0820,0.513,80.030,248480,4,0.94
0,1PuLHwFZoh5qYK89I5YBdZ,I Will Wait,Mumford & Sons,0.486,0.769,1,-5.140,1,0.0119,0.00832,0.1970,0.393,131.286,276720,4,0.42


In [35]:
print("How similar is it to my taste?", round(felipe_df['prediction'].mean()*100,3), "%")

How similar is it to my taste? 83.932 %
